<h1 align="center"> Accessing Astronomical Data From Python </h1>
<h2 align="center"> Part 2: Accessing and Visualizing Astronomical Data </h2> 

## 0.- Installing necessary packages
* ``conda install astropy``
* ``conda install -c astropy aplpy astroquery``

Before doing to data analysis, we need to find, load and pre-process data!

## 1.- Astronomical Data from WebPages

Function to retrieve the tarball and the fits file.

In [ ]:
import os
import urllib.request
import tarfile
import sys

global DOWNLOADS_DIR 
DOWNLOADS_DIR= '.'

def download_and_extract(url):
    # Obtain the filename
    name = url.rsplit('/', 1)[-1]
    filename = os.path.join(DOWNLOADS_DIR, name)
    # Download the file if not found
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)
    # Decompress (if needed) and copy the selected file to the DOWNLOADS_DIR
    sdir=filename.rsplit('ReferenceImages',1)[0]+"ReferenceImages"
    if not os.path.isdir(sdir):
        tar = tarfile.open(filename)
        tar.extractall(path=DOWNLOADS_DIR)
        tar.close()

In [ ]:
# Data from ALMA science verification 
twhydra_url = 'https://almascience.nrao.edu/almadata/sciver/TWHya/TWHYA_BAND7_ReferenceImages.tgz'
download_and_extract(twhydra_url)

!ls TWHYA_BAND7_ReferenceImages

## 2.- Loading and a Quick Look at the Data

In [ ]:
from astropy.io import fits
import logging
import warnings
warnings.filterwarnings('ignore')

# Load File
hdulist = fits.open("TWHYA_BAND7_ReferenceImages/TWHydra_CO3_2line.image.mom0.fits")
twhydra = hdulist[0]

### Show the header

In [ ]:
twhydra.header

### Quick Look
We can use matplotlib

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(twhydra.data.sum(axis=(0,1)))

In [ ]:
import aplpy 
fig = aplpy.FITSFigure(twhydra)
fig.show_grayscale()
fig.show_grid()
fig.show_contour()

In [ ]:
#Load Data Cube
hdulist = fits.open("TWHYA_BAND7_ReferenceImages/TWHydra_CO3_2line.image.fits")
twhydra_cube = hdulist[0]

def show_spectra(cube):
    spec = cube.sum(axis=(0,2,3))
    plt.plot(spec)
    plt.ylabel('Flux')
    plt.xlabel('Channel')

show_spectra(twhydra_cube.data)
plt.show()

## Interactivity

In [ ]:
from ipywidgets import interact

def interactive_denoising(threshold=0.0):
    cube=twhydra_cube.data.copy()
    cube[cube<threshold] = 0
    show_spectra(cube)
    plt.show()

mmax = twhydra_cube.data.max()
mmin = twhydra_cube.data.min()
print(mmax,mmin)
res=interact(interactive_denoising,threshold=(mmin,mmax))

In [ ]:
limits = res.widget.kwargs
limits

In [ ]:
cube=twhydra_cube.data[0]
def explore_channels(chan=0):
    plt.imshow(cube[chan])
    plt.colorbar()
    plt.show()

interact(explore_channels,chan=(0,cube.shape[0]-1))

## 3 Catalog Access

In [32]:
from astroquery.skyview import SkyView
dd=SkyView.list_surveys()

{'All:GOODS/HDF/CDF': ['GOODS: Chandra ACIS HB',
                       'GOODS: Chandra ACIS FB',
                       'GOODS: Chandra ACIS SB',
                       'GOODS: VLT VIMOS U',
                       'GOODS: VLT VIMOS R',
                       'GOODS: HST ACS B',
                       'GOODS: HST ACS V',
                       'GOODS: HST ACS I',
                       'GOODS: HST ACS Z',
                       'Hawaii HDF U',
                       'Hawaii HDF B',
                       'Hawaii HDF V0201',
                       'Hawaii HDF V0401',
                       'Hawaii HDF R',
                       'Hawaii HDF I',
                       'Hawaii HDF z',
                       'Hawaii HDF HK',
                       'GOODS: HST NICMOS',
                       'GOODS: VLT ISAAC J',
                       'GOODS: VLT ISAAC H',
                       'GOODS: VLT ISAAC Ks',
                       'HUDF: VLT ISAAC Ks',
                       'GOODS: Spitzer IRAC 3

In [33]:
res=SkyView.get_image_list('M100',['NVSS'])
print(res)

['https://skyview.gsfc.nasa.gov/tempspace/fits/skv8641792759056.fits']


In [ ]:
urllib.request.urlretrieve(res[0],'opt.fits')
hdulist = fits.open("opt.fits")
opt = hdulist[0] 
fig = aplpy.FITSFigure(opt)
fig.show_grayscale()

In [ ]:

from astropy.wcs import WCS
from matplotlib.patches import Circle
ra=opt.header['CRVAL1']
dec=opt.header['CRVAL2']
mywcs=WCS(opt.header)
max_radius=0.09

def select_radius(radius=max_radius):
    ax=plt.subplot(111,projection=mywcs)
    ax.imshow(opt.data,cmap="gray",origin='lower')
    cc=Circle((ra,dec),radius=radius,edgecolor="green",facecolor="none",transform=ax.get_transform('fk5'))
    ax.add_patch(cc)
    
mycirc=interact(select_radius,radius=(0.0,max_radius,max_radius/1000))

In [ ]:
limits = mycirc.widget.kwargs
rad    = limits['radius']
print(ra,dec)
rad

In [ ]:
from astroquery.vizier import Vizier
catalog_list = Vizier.find_catalogs(keywords=['GAIA'])
for key,item in catalog_list.items():
    print(key+" : "+item.description)

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

c = SkyCoord(ra=ra*u.deg, dec=dec*u.deg)
res=Vizier.query_region(c, radius=rad*u.deg, catalog='I/337')

In [ ]:
restab=res[0]
restab.show_in_notebook()
#restab

In [ ]:
fig = aplpy.FITSFigure(opt)
fig.show_grayscale()
#fig.show_markers(restab['ra'],restab['dec'])
fig.show_markers(restab['RA_ICRS'],restab['DE_ICRS'])

### With ALMA

In [ ]:
res=SkyView.get_image_list(position='Galactic Center',survey=['NVSS'])
res

In [ ]:
%matplotlib inline
urllib.request.urlretrieve(res[0],'opt.fits')
hdulist = fits.open("opt.fits")
opt = hdulist[0] 
fig = aplpy.FITSFigure(opt)
fig.show_grayscale()

In [ ]:
mycirc=interact(select_radius,radius=(0.0,1,1.0/1000))

In [ ]:
limits = mycirc.widget.kwargs
ra=opt.header['CRVAL1']
dec=opt.header['CRVAL2']
rad    = limits['radius']
print(ra,dec)
rad

In [ ]:
from astropy.vo.client import vos_catalog
catalog=vos_catalog.VOSCatalog.create("ChiVO ALMA",
                                      "http://vo.chivo.cl/archive_alma/q/scs-archive-alma/scs.xml?")
catalog.dumps()

In [ ]:
res=vos_catalog.call_vo_service('conesearch_good',
                            catalog_db=catalog,
                            kwargs={'RA': ra, 'DEC': dec,'SR': rad})

In [ ]:
almatab=res.to_table()
almatab.show_in_notebook()

In [ ]:
%matplotlib notebook
fig = aplpy.FITSFigure(opt)
fig.show_grayscale()
fig.show_markers(almatab['s_ra'],almatab['s_dec'])

In [ ]:
# http://vo.chivo.cl:9000/tarball?asdm=uid://A002/Xa657ad/Xd3b&mous=uid://A001/X122/X2b